# Movie Review Analysis Outline
## import necessary python libraries


In [8]:
# output.txt is created by running a python script to convert the format of the movie data
# into a format that spark readily converts into a RDD

# "transformData.py" is this script which is intended to be run once on our data
import re
import string
from operator import add
import os
import sys
from pyspark.mllib.feature import HashingTF
from pyspark.mllib.regression import LabeledPoint

import pyspark.mllib.regression as mllib_reg
import pyspark.mllib.linalg as mllib_lalg
import pyspark.mllib.classification as mllib_class
import pyspark.mllib.tree as mllib_tree
print (pyspark) # test to see that pyspark is up and running okay



<module 'pyspark' from '/usr/local/bin/spark-1.3.1-bin-hadoop2.6/python/pyspark/__init__.py'>


# CLEAN DATA

In [9]:
def strip_html_tags(data):
    p = re.compile(r'<.*?>')
    return p.sub('', data.lower())

#parse and take care of funky symbols in summary and text <- normalize to root words maybe?
def cleanData (data):
    data = strip_html_tags(data)
    data =  re.sub("[\t\,\:;\(\)\"\'\~\-\!\?\`]", "",data, 0, 0)
    data =  re.sub("[\.0`]", "",data, 0, 0) # special case to get rid of ".0" of scores -- 
    return data

#re.sub("[\.\t\,\:;\(\)\.]", " ", strip_html_tags(data.lower()), 0, 0)

# clean the data for each element of each sub list
def prepData (list_str):
    L= []
    for x in list_str:
        #print x
        L.append(cleanData(x).strip())
    return L

# [u'productId', u'userId', u'profileName', u'helpfulness', u'score', u'time', u'summary', u'text']
# headers for the project


# PARSE THE FILE WE WANT TO USE AS OUR DATA

In [10]:

# CHANGED SEPARATOR TO THREE /// because / separator was causing discrepencies
# the original file for the current output.txt is a truncated version of all our data, so the end elements will be funky

# output < smaller_parsed
#movies_txt = sc.textFile("output.txt").map(lambda x: (cleanData(x).split('///')))
movies_txt2 = sc.textFile("smaller_parsed.txt").map(lambda x: (cleanData(x).split('///')))

In [11]:
'''
trying to parse without using a separate python file (wihtout using transformData.py)


# CHANGED SEPARATOR TO THREE /// because / separator was causing discrepencies
# the original file for the current output.txt is a truncated version of all our data, so the end elements will be funky


######
# So our first issue, the transformData.py script cannot read more than 100k data points in one go...
# crashs afterwards, so no where near reading the entire 7 mil we need 
# HOW CAN WE READ THE 7 million data points and group every group of 8 datapoints together without crashing any servers
#### option 1 - try something with spark 
##### -> for now we will test ~111k data points with several machine learning techniques and then get back to this problem if we have time
#####



# movies_txt = sc.textFile("movies.txt").map(lambda x: (cleanData(x)))

# smaller_movies is the head 10000k bytes of the original movie data set
movies_txt = sc.textFile("smaller_movies.txt").map(lambda x: (lambda y: cleanData(" ".join(y.strip().split(" ")[1:]))) (x))



#movies_txt = sc.textFile("tiniest_movies.txt").map(lambda x: (lambda y: cleanData(" ".join(y.strip().split(" ")[1:]))) (x))

# DONT TRY TO PRINT ALL THE MOVIE REVIEWS... it WILL CRASH THE KERNEL
# get through parsing the data and sectioning everything off before trying (or better yet, just dont try... seriously )
# big data = BIG and LARGE and SLOW


#print movies_test.collect()
movies_txt1 = movies_txt.filter(lambda x: len(x) > 0 and x.strip() != '' ) # remove all empty strings from the rdd
movies_txt2 = movies_txt1.zipWithIndex().groupBy(lambda (string, idx): idx/8 )\
        .map(lambda(id_num, str1):  tuple([string  for string, idx in str1  ])  )

#movies_final_txt = movies_txt2.count()
print movies_txt2.count()
#print movies_txt2.collect()


#### USING PARSED FILES from transformData.py
# output < smaller_parsed
#movies_txt = sc.textFile("output.txt").map(lambda x: (cleanData(x).split('///')))
'''

'\ntrying to parse without using a separate python file (wihtout using transformData.py)\n\n\n# CHANGED SEPARATOR TO THREE /// because / separator was causing discrepencies\n# the original file for the current output.txt is a truncated version of all our data, so the end elements will be funky\n\n\n######\n# So our first issue, the transformData.py script cannot read more than 100k data points in one go...\n# crashs afterwards, so no where near reading the entire 7 mil we need \n# HOW CAN WE READ THE 7 million data points and group every group of 8 datapoints together without crashing any servers\n#### option 1 - try something with spark \n##### -> for now we will test ~111k data points with several machine learning techniques and then get back to this problem if we have time\n#####\n\n\n\n# movies_txt = sc.textFile("movies.txt").map(lambda x: (cleanData(x)))\n\n# smaller_movies is the head 10000k bytes of the original movie data set\nmovies_txt = sc.textFile("smaller_movies.txt").map(


## Parsing out relevant fields - score and text of review

In [14]:
# prep our data

#print movies_txt2.top(1)
movies_new = movies_txt2.map(lambda L: (L[0], L[4], L[6], L[7]) if len(L) == 8 else L )
movies_new1 = movies_new.filter(lambda L: len(L) == 4)
# check that we aren't losing any data 
# --> we are still missing some from our parsing.... :/ 
# 9 in fact... we can fix this later

print (movies_new1.count())
print (movies_new.count() == movies_new1.count())

removeHTMLTags = movies_new1.map(prepData) # remove html tags

111101
False


## REVISED STOP WORDS REMOVAL 

In [15]:
#REVISED STOP WORDS REMOVAL Approach

#used the stopwords file in the virtual box instead... taking too long
baseDir = os.path.join('../data')
inputPath = os.path.join('cs100', 'lab3')
STOPWORDS_PATH = 'stopwords.txt'
split_regex = r'\W+'

stopfile = os.path.join(baseDir, inputPath, STOPWORDS_PATH)
stopwords = set(sc.textFile(stopfile).collect())
#print 'These are the stopwords: %s' % stopwords

def tokenize(string):
    #reusing assignment code
    """ An implementation of input string tokenization to exclude stopwords
    Args:
        string (str): input string
    Returns:
        list: a list of tokens without stopwords
    """
    splitList = re.split(split_regex, string.lower().strip())
    
    return [x for x in splitList if x!="" and x not in stopwords]  


intSW = removeHTMLTags.map(lambda x: x[1]).top(20)
print intSW
removeSW_output= removeHTMLTags.map(lambda x: (  int((str(x[1])).replace("/","")), tokenize(x[3]))   )
#print(removeSW_output.collect())
#print (removeSW_output.count())

[u'5', u'5', u'5', u'5', u'5', u'5', u'5', u'5', u'5', u'5', u'5', u'5', u'5', u'5', u'5', u'5', u'5', u'5', u'5', u'5']


## Vectorizing - making the vector of features...

In [16]:
## Took forver to figure out
## key notes, use mllib if you are using rdd, ditch ml if you are not using data frams

htf = HashingTF(numFeatures=1) # features need to match dimensions of output

print "just testing the hashing TF transformation on the array: ['hi', 'boo', 'hi']"
print (htf.transform(["hi", "boo", "hi"]));
feature_vector = removeSW_output.map(lambda elements: LabeledPoint(1.0, [htf.transform(elements[1])]) if float(elements[0]) >= 2.5 else LabeledPoint(0.0, [htf.transform(elements[1])]))

#feature_vector = removeSW3.map(lambda elements: LabeledPoint(1, [2]))
print feature_vector.filter(lambda x: x.label == 1.0).count()

just testing the hashing TF transformation on the array: ['hi', 'boo', 'hi']
(1,[0],[3.0])
95309


In [21]:
# EXPLORE DIFFERENT ML APPROACHS
# https://spark.apache.org/docs/1.1.0/api/python/pyspark.mllib.classification-module.html

##  Trying SVM approach

In [20]:
from pyspark.mllib.classification import SVMWithSGD, SVMModel, LogisticRegressionWithLBFGS 
from pyspark.mllib.regression import LabeledPoint


'''  REFRESH BLOCK
REFRESHING DATA
Re Run data -> refresh in case we are using another approach above this as well
Things to do - update number of features
'''
## Took forever to figure out
## key notes, use mllib if you are using rdd, ditch ml if you are not using data frams
htf = HashingTF(numFeatures=1) # features need to match dimensions of output
print ("SVM: REFRESHING HTF, and FEATURE VECTOR")
feature_vector = removeSW_output.map(lambda elements: LabeledPoint(1.0, [htf.transform(elements[1])]) if float(elements[0]) >= 2.5 else LabeledPoint(0.0, [htf.transform(elements[1])]))
'''
REFRESHED DATA
'''


training_data, testing_data= feature_vector.randomSplit([0.7, 0.3])


#LogisticRegressionWithLBFGS 
# Build the Support Vector Machine model
model = SVMWithSGD.train(training_data, iterations=100) # lower iterations so we arent here forever

# Evaluate the model on training data
labelsAndPreds = training_data.map(lambda p: (p.label, model.predict(p.features)))
trainErr = labelsAndPreds.filter(lambda (v, p): v != p).count() / float(training_data.count())
print("training error of the SVM model= " + str(trainErr))




#Evaluate on testing data.. the correct way lolz

labelsAndPreds = testing_data.map(lambda p: (p.label, model.predict(p.features)))
trainErr = labelsAndPreds.filter(lambda (v, p): v != p).count() / float(testing_data.count())
print("testing error of the SVM model= " + str(trainErr))



###### TO DO 
#labelsAndPreds = feature_vector.map(lambda p: (p.label, model.predict(p.features)))
#trainErr = labelsAndPreds.filter(lambda (v, p): v != p).count() / float(feature_vector.count())
#print("training error of the model= " + str(trainErr))

# Saving the model is meh
#model.save(sc, "target/tmp/pythonSVMWithSGDModel")
#sameModel = SVMModel.load(sc, "target/tmp/pythonSVMWithSGDModel")

SVM: REFRESHING HTF, and FEATURE VECTOR
training error of the SVM model= 0.140778324362
testing error of the SVM model= 0.145304511615


In [22]:
from pyspark.mllib.classification import SVMWithSGD, SVMModel, LogisticRegressionWithLBFGS 
from pyspark.mllib.regression import LabeledPoint


'''  REFRESH BLOCK
REFRESHING DATA
Re Run data -> refresh in case we are using another approach above this as well
Things to do - test/update number of features
'''
htf = HashingTF(numFeatures=1) # features need to match dimensions of output
print ("LogisticRegressionWithLBFGS: REFRESHING HTF, and FEATURE VECTOR")
feature_vector = removeSW_output.map(lambda elements: LabeledPoint(1.0, [htf.transform(elements[1])]) if float(elements[0]) >= 2.5 else LabeledPoint(0.0, [htf.transform(elements[1])]))
'''
REFRESHED DATA
'''

training_data, testing_data= feature_vector.randomSplit([0.7, 0.3])

#LogisticRegressionWithLBFGS 
# Build the Support Vector Machine model
model = LogisticRegressionWithLBFGS.train(training_data, iterations=100) # lower iterations so we arent here forever

# Evaluate the model on training data
labelsAndPreds = training_data.map(lambda p: (p.label, model.predict(p.features)))
trainErr = labelsAndPreds.filter(lambda (v, p): v != p).count() / float(training_data.count())
print "Log Training Accuracy: " + "%.3f" %(100*(1-trainErr)) + "%" 
print("training error of the LogisticRegressionWithLBFGS model= " + "%.3f" % (100*(trainErr))) + "%" 

#Evaluate on testing data.. the correct way lolz

labelsAndPreds = testing_data.map(lambda p: (p.label, model.predict(p.features)))
testErr = labelsAndPreds.filter(lambda (v, p): v != p).count() / float(testing_data.count())

print "Log Testing Accuracy: " + "%.3f" %(100*(1-testErr)) + "%" 
print("testing error of the LogisticRegressionWithLBFGS model= " + "%.3f" %(testErr*100)) + "%" 


LogisticRegressionWithLBFGS: REFRESHING HTF, and FEATURE VECTOR
Log Training Accuracy: 85.651%
training error of the LogisticRegressionWithLBFGS model= 14.349%
Log Testing Accuracy: 86.100%
testing error of the LogisticRegressionWithLBFGS model= 13.900%


## Do some Predictions - Bayes

In [23]:
### THIS USES NAIVE BAYES CLASSIFIER
### with 116 elements with the first output.txt 86KB , gets 80-93%
# tested with lots of data.... - 111MB

'''  REFRESH BLOCK
REFRESHING DATA
Re Run data -> refresh in case we are using another approach above this as well
Things to do - update number of features
'''
## Took forever to figure out
## key notes, use mllib if you are using rdd, ditch ml if you are not using data frams
htf = HashingTF(numFeatures=1) # features need to match dimensions of output
print ("BAYES: REFRESHING HTF, and FEATURE VECTOR")
feature_vector = removeSW_output.map(lambda elements: LabeledPoint(1.0, [htf.transform(elements[1])]) if float(elements[0]) >= 2.5 else LabeledPoint(0.0, [htf.transform(elements[1])]))
'''
REFRESHED DATA
'''

# would be nice to figure out which division is the best
training_data, testing_data= feature_vector.randomSplit([0.6, 0.4])
#training_data, testing_data= feature_vector.randomSplit([0.7, 0.3])

# parameters:
lamda = 1.0

# build Naive Bayes Classifier
nbay = mllib_class.NaiveBayes.train(training_data, lamda)

# Make prediction and test accuracy.
predictionAndLabel = testing_data.map(lambda p : (nbay.predict(p.features), p.label))
testErr = predictionAndLabel.filter(lambda (v, p): v != p).count() / float(testing_data.count())
accuracy = 100.0 * predictionAndLabel.filter(lambda (x, v): x == v).count() / testing_data.count()

print "Naive Bayes Accuracy: " + "%.3f" %(accuracy) + "%" 
print "Bayes error: " + "%.3f" % (100-accuracy) + "%" 

BAYES: REFRESHING HTF, and FEATURE VECTOR
Naive Bayes Accuracy: 85.809%
Bayes error: 14.191%
